## CUR Decomposition

In [1]:
import pandas as pd
import os
from pathlib import Path
import scipy
import pickle
import scipy.stats
from numpy import *
import numpy as np
from numpy import linalg as la
from sklearn.metrics import mean_squared_error
root = Path(".")
from datetime import datetime

CUR decomposition is similar to SVD in a way that it also decomposes a given matrix into three different matrices C, U and R where R represents the diagonal matrix but it differs from SVD in the method through which C, U and R are formed
</br> </br>
Formally, a CUR matrix approximation of a matrix A is three matrices C, U, and R such that C is made from columns of A, R is made from rows of A, and that the product CUR closely approximates A. Usually the CUR is selected to be a rank-k approximation, which means that C contains k columns of A, R contains k rows of A, and U is a k-by-k matrix.
</br> </br>
Each row and column of A has some probability assigned to it which is equal to sum of squares of all entries in that row divided by sum of squares of all elements in the matrix. K rows and and K columns are sampled from the original matrix with respect to the probability that any row/column is picked proportional to sum of squares of entries it has.
</br></br>
Once the matrix is decomposed, the method of predicting the ratings is similar to SVD.

In [2]:
class CUR:
    def __init__(self, retained_energy) -> None:
        self.read_dataset()
        self.train_test_split()
        self.generate_user_item_matrix()
        self.U, self.S, self.V_T = self.CUR_decomp(500)
        self.CUR_transform(retained_energy)
        self.calculate_item_similarity()
        
        self.sim_top_k = [None for _ in range(self.item_cnt + 1)]
        for i in range(1, self.item_cnt): 
            self.sim_top_k[i] = self.get_sim_items(i)

    def CUR_decomp(self , red_dim): 
        C = np.zeros((self.matrix.shape[0], red_dim))
        R = np.zeros((red_dim, self.matrix.shape[1]))
        sum_of_squares = np.sum(self.matrix ** 2)
        probability_colwise = np.sum(self.matrix ** 2, axis=0) / sum_of_squares
        col_ids = np.arange(self.matrix.shape[1])

        taken_cols = 0
        taken_col_list = list()
        dup_col_list = list()
        
        while(taken_cols < red_dim) : 
            cur_p = np.random.choice(col_ids, p = probability_colwise)
            if cur_p not in taken_col_list : 
                C[:, taken_cols] = self.matrix[:, cur_p] / np.sqrt(probability_colwise[cur_p] * red_dim)
                taken_cols += 1
                taken_col_list.append(cur_p)
                dup_col_list.append(1)
            else : 
                get_id = taken_col_list.index(cur_p)
                dup_col_list[get_id] += 1
        
        C = np.multiply(C, np.sqrt(dup_col_list))

        sum_of_squares_row = np.sum(self.matrix ** 2, axis = 1)
        probability_row_wise = sum_of_squares_row / sum_of_squares
        col_ids = np.arange(self.matrix.shape[0])
        taken_row = 0
        taken_row_list = list()
        dup_row_list = list()

        while(taken_row < red_dim) : 
            cur_p = np.random.choice(col_ids, p = probability_row_wise)
            if cur_p not in taken_row_list : 
                R[taken_row, :] = self.matrix[cur_p, :] / np.sqrt(probability_row_wise[cur_p] * red_dim)
                taken_row += 1
                taken_row_list.append(cur_p)
                dup_row_list.append(1)
            else : 
                get_id = taken_row_list.index(cur_p)
                dup_row_list[get_id] += 1
        
        R = np.multiply(R.T, np.sqrt(dup_row_list)).T

        Sigma = np.zeros((red_dim, red_dim))
        for i, I in enumerate(taken_row_list): 
            for j, J in enumerate(taken_col_list):
                Sigma[i, j] = self.matrix[I, J]
        
        cur_U, cur_S, cur_V_T = la.svd(Sigma, red_dim)
        new_S = np.zeros((red_dim, red_dim))
        for i in range(red_dim):
            new_S[i, i] = cur_S[i]

        cur_S = new_S 
        for cols in range(red_dim):
            if cur_S[cols, cols] >= 1:
                cur_S[cols, cols] = 1 / cur_S[cols, cols]
            else: 
                cur_S[cols, cols] = 0
        
        cur_S_sq = np.dot(cur_S, cur_S)
        U = np.dot(cur_V_T.T, np.dot(cur_S_sq, cur_U.T))
        print(U.shape)
        return C, U, R

    def read_dataset(self):
        self.rating_data = pd.read_csv('ml-1m/ratings.dat', header=None, sep='::')
        self.rating_data.columns = ['UserID', 'ItemID', 'Rating', 'Timestamp']
        self.rating_data.drop(columns=['Timestamp'], axis=1, inplace=True)
        
        self.item_cnt = max(self.rating_data['ItemID']) + 1
        self.users_cnt = max(self.rating_data['UserID']) + 1

    def train_test_split(self):
        self.rating_train = self.rating_data.sample(frac=0.8, random_state=200)
        self.rating_test = self.rating_data.drop(self.rating_train.index)

    def generate_user_item_matrix(self):
        self.matrix = np.zeros(shape=(self.users_cnt, self.item_cnt))
        for row in self.rating_train.itertuples():
            self.matrix[row.UserID][row.ItemID] = row.Rating

    def get_dim(self, S, retained_energy): 
        sum_sq = 0
        val_list = list()
        for i in range(S.shape[0]) : 
            sum_sq += S[i, i] ** 2
            val_list.append(S[i, i])
        cur_sum, cur_d = 0, 0
        for val in val_list:
            if cur_sum >= sum_sq * retained_energy: 
                return cur_d 
            cur_sum += val ** 2
            cur_d += 1
        return cur_d

    def similarity(self, A, B):
        return (1.0 / (1.0 + la.norm(A - B)))

    def CUR_transform(self, retained_energy): 
        red_dim = self.get_dim(self.S, retained_energy)

        transformed_S = np.diag(self.S[:red_dim])
        transformed_U = self.U[:, :red_dim]
        transformed_V_T = self.V_T[:red_dim, :]
        new_S = np.zeros((transformed_S.shape[0] , transformed_S.shape[0]))
        print(new_S.shape)
        for i in range(transformed_S.shape[0]) : 
            new_S[i, i] = transformed_S[i]
        transformed_S = new_S
        print(transformed_S.shape , transformed_U.shape , transformed_V_T.shape)
        self.SVD_matrix = np.dot(transformed_U, transformed_S)
        self.SVD_matrix = np.dot(self.SVD_matrix, transformed_V_T)
    
    def calculate_item_similarity(self):
        self.item_similarity = np.zeros(shape=(self.item_cnt, self.item_cnt))
        transformed_item_matrix = list()
        for items in range(1, self.item_cnt): 
            transformed_item_matrix.append(self.SVD_matrix[items, :].T) 

        for item1 in range(1, self.item_cnt): 
            for item2 in range(item1 + 1, self.item_cnt): 
                sim = self.similarity(transformed_item_matrix[item1 - 1], transformed_item_matrix[item2 - 1])
                
                self.item_similarity[item1][item2] = self.item_similarity[item2][item1] = sim
    
    def get_sim_items(self, itemID): 
        sim_userID = list()
        for item in range(1, self.item_similarity.shape[0]): 
            if item == itemID: 
                continue
            sim_userID.append((self.item_similarity[itemID][item], item))
        sim_userID.sort(key=lambda y: -y[0])
        return sim_userID

    def predict_SVD(self, userID, itemID, top_K):
        similarity_sum = 0 
        rating_sum, cnt = 0, 0
        
        for (item_s, items) in self.sim_top_k[itemID]:
            if cnt == top_K: 
                break
            if self.matrix[userID][items] == 0: 
                continue
            similarity_sum += item_s
            rating_sum += item_s * self.matrix[userID][items]
            cnt += 1
        if similarity_sum == 0:
            return 0
            
        return (rating_sum / similarity_sum)
    
    def get_top_k(self, k=3):
        start_prediction_time = datetime.now()
        for users in range(1, self.users_cnt): 
            unrated_items = np.where(self.matrix[users] == 0)[0]
            for items in unrated_items:
                if items == 0:
                    continue
                assert(self.matrix[users][items] == 0)
                self.matrix[users][items] = self.predict_SVD(users, items, k)
        end_prediction_time = datetime.now()
        print("Time for prediction" , (end_prediction_time - start_prediction_time).total_seconds())

    def get_rmse(self):
        y_actual = list(self.rating_test.Rating)
        y_pred = list()
        for id, row in self.rating_test.iterrows(): 
            uid, itid = row['UserID'], row['ItemID']
            y_pred.append(self.matrix[uid][itid])
        return mean_squared_error(y_actual, y_pred) ** .5

In [3]:
cur_recom = CUR(0.9)
cur_recom.get_top_k()
cur_recom.get_rmse()

my_path = root / "Pickled_files" / "CUR90"
dbfile = open(my_path, 'wb')     
pickle.dump(cur_recom.matrix,dbfile)
dbfile.close()

C:\Users\ashwi\AppData\Local\Temp\ipykernel_11768\668386011.py:81: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  self.rating_data = pd.read_csv('ml-1m/ratings.dat', header=None, sep='::')


(500, 500)
(437, 437)
(437, 437) (6041, 437) (437, 3953)
Time for prediction 223.776183


In [4]:
print(cur_recom.get_rmse())

1.1851374361914413


In [5]:
cur_recom = CUR(1.0)
cur_recom.get_top_k()


my_path = root / "Pickled_files" / "CUR100"
dbfile = open(my_path, 'wb')     
pickle.dump(cur_recom.matrix,dbfile)
dbfile.close()

C:\Users\ashwi\AppData\Local\Temp\ipykernel_11768\668386011.py:81: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  self.rating_data = pd.read_csv('ml-1m/ratings.dat', header=None, sep='::')


(500, 500)
(500, 500)
(500, 500) (6041, 500) (500, 3953)
Time for prediction 220.19741


In [6]:
print(cur_recom.get_rmse())

1.1781062652249525
